In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

Compare values from the MVP with the existing logger
--

Data from the existing logger is in

```waihaha.gns.cri.nz:/home/sysop/csi/inferno/2018``` with files like ```2018.034.Inferno-utc.csv```

In [ ]:
import pandas as pd
%matplotlib inline

**Data files**

In [ ]:
mvpfile = 'Table1.csv'
dtstart = '2017-12-06T00:30:00'

existfile = 'existing_logger.txt'

**Read data**

In [ ]:
#MVP
df = pd.read_csv(mvpfile, parse_dates=True, index_col = 'Datetime')
#Use only data after logger was installed
mvp = df[df.index>=dtstart]

In [ ]:
mvp.tail()

**Get recent existing data**

In [ ]:
%%bash
ssh volcano@waihaha "cat /home/sysop/csi/inferno/2018/2018.???.Inferno-utc.csv" > existing_logger_2018.txt

In [ ]:
%%bash
cat existing_logger_2017.txt existing_logger_2018.txt > existing_logger.txt
#drop header lines
sed -i '/^I/ d; /^T/ d; /^C/ d' existing_logger.txt

In [ ]:
#existing
names = ['date', 'time', 'temp', 'level']
exist = pd.read_csv(existfile, sep='\s+', skiprows=3, names=names, parse_dates={"Datetime" : ['date', 'time']}, usecols=[0,1,5,8], index_col = [0])

In [ ]:
exist.tail()

**Resample existing data on same time scale as MVP**

In [ ]:
exist1 = exist.resample('10T')
exist2 = exist1.interpolate(method='linear')

In [ ]:
exist2.tail()

**Temperature**

In [ ]:
tmp = mvp['Temp_PRT_degC'].plot(figsize=(20,7), title='Temperature', label='MVP', fontsize=12)
exist2['temp'].plot(ax=tmp, label='existing, interpolated')
tmp.title.set_size(20)
tmp.grid()
tmp.set_ylabel("Temperature (deg C)")
tmp.set_ylim(ymin=10, ymax=80)
tmp.legend(loc='best')

In [ ]:
fig = tmp.get_figure()
fig.savefig('temperature_compare.png', dpi=100)

**Temperature difference**

In [ ]:
diff = mvp['Temp_PRT_degC'] - exist2['temp']
diff.describe()

**Depth**

In [ ]:
diff = mvp['Depth_USGS_OTT_meters'] - exist2['level']
diff.mean()
exist2['level'] += diff.mean()

In [ ]:
dep = mvp['Depth_USGS_OTT_meters'].plot(figsize=(20,7), title='Water Depth Above Sensor (existing is corrected for comparison)',fontsize=12, label='MVP')
exist2['level'].plot(ax=dep, label='existing, interpolated')
dep.title.set_size(20)
dep.grid()
dep.set_ylabel("Depth (m)")
dep.legend(loc='best')

In [ ]:
fig = dep.get_figure()
fig.savefig('depth_compare.png', dpi=100)